<a href="https://colab.research.google.com/github/LucasEPrz/APRENDIZAJE-POR-REFUERZO/blob/main/Lab2/Lab2_Perez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Actividad 1**

Crear tu propio entorno y entrenar agentes RL en el mismo. Analizar la convergencia con distintos algoritmos* (ej: PPO, DQN), resultados con distintas funciones de recompensa e híper-parámetros.

Algunas ideas:

Transformar GoLeftEnv en una grilla 2D, añadir paredes / trampas / agua.
Crear un entorno que juegue a algún juego como el ta-te-ti.
Crea un entorno totalmente nuevo que sea de tu interés!

# **Creación del entorno**

Definir la estructura del entorno

In [ ]:
import gym
from gym import spaces
import numpy as np

class CustomGridEnv(gym.Env):
    def __init__(self):
        super(CustomGridEnv, self).__init__()

        self.grid_size = 5  # Tamaño de la grilla (5x5)
        self.agent_pos = [0, 0]  # Posición inicial del agente (en la esquina superior izquierda)
        self.goal_pos = [4, 4]  # Posición del objetivo (en la esquina inferior derecha)

        # Definir la acción: [arriba, abajo, izquierda, derecha]
        self.action_space = spaces.Discrete(4)

        # Definir el espacio de observación: matriz 5x5, donde 0 = vacío, 1 = pared, 2 = trampa, 3 = objetivo
        self.observation_space = spaces.Box(low=0, high=3, shape=(self.grid_size, self.grid_size), dtype=int)

        # Mapa con paredes y trampas
        self.grid = np.zeros((self.grid_size, self.grid_size))
        self.grid[1, 1] = 1  # Pared en (1,1)
        self.grid[2, 2] = 2  # Trampa en (2,2)

    def reset(self):
        self.agent_pos = [0, 0]  # Reiniciar la posición del agente
        return self.grid  # Devuelve el estado inicial de la grilla

    def step(self, action):
        # Mover al agente en función de la acción
        if action == 0:  # Arriba
            self.agent_pos[0] = max(0, self.agent_pos[0] - 1)
        elif action == 1:  # Abajo
            self.agent_pos[0] = min(self.grid_size - 1, self.agent_pos[0] + 1)
        elif action == 2:  # Izquierda
            self.agent_pos[1] = max(0, self.agent_pos[1] - 1)
        elif action == 3:  # Derecha
            self.agent_pos[1] = min(self.grid_size - 1, self.agent_pos[1] + 1)

        # Verificar si el agente ha llegado al objetivo o ha caído en una trampa
        done = False
        reward = -1  # Penalización por cada paso
        if self.agent_pos == self.goal_pos:
            reward = 10  # Recompensa por llegar al objetivo
            done = True
        elif self.grid[self.agent_pos[0], self.agent_pos[1]] == 2:  # Si cae en una trampa
            reward = -10  # Recompensa negativa por caer en una trampa
            done = True

        return self.grid, reward, done, {}

    def render(self):
        # Imprimir la grilla para visualizar el entorno
        grid_copy = self.grid.copy()
        grid_copy[self.agent_pos[0], self.agent_pos[1]] = 5  # Representar al agente con un 5
        print(grid_copy)


# **Entrenar el agente con un algoritmo de RL**

In [2]:
!pip install 'shimmy>=2.0'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 15.8 MB/s eta 0:00:00


In [3]:
!pip install stable-baselines3[extra]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.7 MB/s eta 0:00:00


# Entrenar el agente con PPO

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

# Crear el entorno
env = CustomGridEnv()

# Vectorizar el entorno
env = DummyVecEnv([lambda: env])

# Crear y entrenar el agente PPO
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device
-----------------------------
| time/              |      |
|    fps             | 1026 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 668        |
|    iterations           | 2          |
|    time_elapsed         | 6          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01902721 |
|    clip_fraction        | 0.31       |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.37      |
|    explained_variance   | 0          |
|    learning_rate        | 0.0003     |
|    loss                 | 13         |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0263    |
|    value_loss           | 75.7       |
----------------------------------------
-----------------------------------

Analizar la convergencia

In [ ]:
model = PPO("MlpPolicy", env, verbose=1, learning_rate=0.001, n_steps=256, gamma=0.99, ent_coef=0.01)

Using cpu device


Evaluación del agente

In [ ]:
state = env.reset()
for _ in range(1000):
    action, _states = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()
    if done:
        break

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/base_vec_env.py:243: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


# Probar con otros algoritmos (DQN)

In [ ]:
from stable_baselines3 import DQN

# Entrenar con DQN
model_dqn = DQN("MlpPolicy", env, verbose=1)
model_dqn.learn(total_timesteps=10000)

Using cpu device
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.578    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1391     |
|    time_elapsed     | 0        |
|    total_timesteps  | 444      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.687    |
|    n_updates        | 85       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 985      |
|    time_elapsed     | 2        |
|    total_timesteps  | 2528     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000332 |
|    n_updates        | 606      |
----------------------------------
----------------------------------
| rollout/            |          |
|  

Graficar la convergencia

In [ ]:
from stable_baselines3.common.vec_env import VecNormalize
from stable_baselines3.common.callbacks import EvalCallback

eval_callback = EvalCallback(env, best_model_save_path='./logs/', log_path='./logs/', eval_freq=5000, deterministic=True, render=False)

# Entrenar el modelo con el callback
model.learn(total_timesteps=10000, callback=eval_callback)



-----------------------------
| time/              |      |
|    fps             | 1187 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 256  |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 846         |
|    iterations           | 2           |
|    time_elapsed         | 0           |
|    total_timesteps      | 512         |
| train/                  |             |
|    approx_kl            | 0.015029857 |
|    clip_fraction        | 0.187       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.02       |
|    explained_variance   | 0           |
|    learning_rate        | 0.001       |
|    loss                 | 28.4        |
|    n_updates            | 70          |
|    policy_gradient_loss | -0.0203     |
|    value_loss           | 57.1        |
-----------------------------------------
----------------------------------

# **Actividad 2**

Entrena agentes en entornos más complejos con librerías de agentes más avanzadas. La idea es que den rienda suelta a su creatividad. Algunas ideas de agentes / entornos:

stable-baselines/rl-baselines-zoo (como lo vimos en este notebook).
CleanRL.
Doom.
Arcade.
Pueden inspirarse en DQN, Policy Gradient o demás secciones del curso de RL de HuggingFace.
También puede ser alguna aplicación o notebook usando Decision transformers (blog de referencia).

In [6]:
!pip install --upgrade stable-baselines3

In [3]:
!pip install gymnasium

  Using cached gymnasium-1.0.0-py3-none-any.whl.metadata (9.5 kB)
Using cached gymnasium-1.0.0-py3-none-any.whl (958 kB)


In [7]:
import gymnasium as gym  # Usa gymnasium en lugar de gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

# Crear y vectorizar el entorno
env = DummyVecEnv([lambda: gym.make("CartPole-v1", render_mode="rgb_array")])

# Crear el modelo PPO
model = PPO("MlpPolicy", env, verbose=1)

# Entrenar el modelo
model.learn(total_timesteps=10000)

Using cpu device
-----------------------------
| time/              |      |
|    fps             | 711  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 609        |
|    iterations           | 2          |
|    time_elapsed         | 6          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00978988 |
|    clip_fraction        | 0.126      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.685     |
|    explained_variance   | -0.00769   |
|    learning_rate        | 0.0003     |
|    loss                 | 8.35       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0211    |
|    value_loss           | 53.6       |
----------------------------------------
-----------------------------------

In [10]:
total_rewards = 0
episodes = 0

obs = env.reset()
for _ in range(1000):
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)

    total_rewards += rewards

    if done:
        episodes += 1
        print(f"Recompensa total del episodio {episodes}: {total_rewards}")
        obs = env.reset()

# Promedio de recompensas por episodio
print(f"Promedio de recompensas por episodio: {total_rewards / episodes}")


Recompensa total del episodio 1: [184.]
Recompensa total del episodio 2: [453.]
Recompensa total del episodio 3: [600.]
Recompensa total del episodio 4: [776.]
Recompensa total del episodio 5: [794.]
Recompensa total del episodio 6: [972.]
Promedio de recompensas por episodio: [166.66667]


# **Conclusiones actividad 2**

En este trabajo, entrené un agente utilizando el algoritmo PPO (Proximal Policy Optimization) en el entorno de CartPole-v1, un clásico problema de control en el que el objetivo es balancear un poste sobre un carrito. Utilicé la librería `Stable-Baselines3` para crear y entrenar el agente, y la evaluación se realizó en un entorno vectorizado con `DummyVecEnv` para simular múltiples entornos de manera eficiente. El entrenamiento consistió en 10,000 pasos de tiempo, donde el agente aprendió a maximizar su recompensa mediante la mejora continua de sus políticas de acción.

Durante la evaluación del modelo, se observó que la recompensa total obtenida por el agente aumentaba con el tiempo. El agente logró recompensas crecientes en cada episodio: comenzó con una recompensa de 184 puntos en el primer episodio y alcanzó un máximo de 972 puntos en el sexto episodio. Este aumento sugiere que el agente fue capaz de aprender y mejorar su desempeño a medida que avanzaba en los episodios.

Sin embargo, el promedio de las recompensas por episodio fue de aproximadamente 166.67, lo que indica que, aunque el agente mejoró su desempeño en algunos episodios, la recompensa total por episodio aún mostró variabilidad. Esto podría ser un indicio de que el agente no ha aprendido una política completamente estable y eficiente en este entorno.

A pesar de que el agente mostró un progreso significativo, el promedio de recompensas por episodio aún está lejos de ser óptimo. Esto podría deberse a que el modelo no ha sido entrenado por un número suficiente de pasos, o a la necesidad de ajustar los hiperparámetros del algoritmo, como la tasa de aprendizaje o la entropía de la política. Además, el entorno de CartPole-v1 es relativamente simple, por lo que la exploración de otros entornos más complejos podría ofrecer más desafíos y una mejor oportunidad para mejorar el rendimiento del agente.

En futuras implementaciones, se podría experimentar con un mayor número de pasos de entrenamiento, la modificación de la arquitectura de la red neuronal o el uso de técnicas adicionales como el ajuste de recompensas o estrategias de exploración más avanzadas.

En conclusión, el agente entrenado con PPO mostró una mejora considerable en el entorno de CartPole-v1, logrando un desempeño razonable en términos de recompensas. Si bien no alcanzó su máximo potencial, los resultados obtenidos proporcionan una base sólida sobre la cual se pueden realizar mejoras. Las futuras investigaciones podrían centrarse en aumentar la capacidad de exploración del agente, ajustar los hiperparámetros y probar con entornos más complejos para observar un comportamiento más robusto y optimizado.